In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
df = pd.read_csv('F1_Claim_Detection_train.csv')
# df = pd.read_csv('/content/aug_train.csv')
from google.colab import drive
drive.mount('/content/drive')
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
df_train = df.sort_values('label',axis = 0) #undersampling
df_train = df.head(2000)
df_train = df.reset_index()

In [ ]:
#splitting training dataset for validation set
df_val = df_train.head(int(len(df_train.index)*0.2))
df_train = df_train.tail(int(len(df_train.index)*0.8))

Preprocessing

In [ ]:
def to_lower(str):
  matches = re.compile(r'.').finditer(str)
  ret_str = ''
  for match in matches:
    ret_str += str[match.start():match.end()].lower()
  return ret_str

"""def rem_punc(str):
  str = re.sub(r'(\.|\?|\,|!|\:|\;|\&|\-|\(|\)|\{|\}|\'|\"|\#|\*|\^|\%|\»)', ' ', str)
  return str"""
def rem_punc(str):
  str = re.sub(r'[^a-z0-9A-Z\s]', ' ', str)
  return str

def rem_url(str):
  str = re.sub(r'(https?\:\/\/)?(www\.)?[A-Za-z0-9][A-Za-z0-9\-]*(\.(com|in|net|co|org|gov|ly|us|ac))+((\/[A-Za-z0-9\_]+)+\.(php|aspx|html)|(\/[A-Za-z0-9\_]*)*)',' ', str)
  return str

def rem_username(str):
    str = re.sub(r'(^|[\s\(\)\.\,\\])\@[a-zA-Z_0-9]+', ' ', str)
    return str

def rem_extraspace(str):
  str = re.sub(r'\s{2,}', ' ', str)
  return str

def rem_alphanum(str):
    str = re.sub(r'(^|.)[0-9]+($|.)', '  ', str)
    return str
"""
def spell_correct(arr):
  spell_correct = Speller(lang = 'en')
  for i in range(len(arr)):
    arr[i] = spell_correct(arr[i])"""

def rem_extra_alpha(str):
    str = re.sub(r'([a-z])\1{2,}', r'\1\1', str)
    return str

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatize(arr):
  lemmatizer = WordNetLemmatizer()
  for i in range(len(arr)):
    arr[i] = lemmatizer.lemmatize(arr[i])

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add('quot')
stop_words.add('amp')
def rem_sw(arr):
  global stop_words
  reg_str = r'^'
  for i in range(len(arr)):
    for sw in stop_words:
      reg_str+=sw
      reg_str+='$'
      arr[i] = re.sub(reg_str, '', arr[i])
      reg_str = r'^'
nltk.download('punkt')
from nltk.tokenize import word_tokenize
def tokenize(str):
    a = word_tokenize(str)
    return a
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
def lemmatize(arr):
    lemmatizer = WordNetLemmatizer()
    for i in range(len(arr)):
        arr[i] = lemmatizer.lemmatize(arr[i])
    return arr
def rem_empty(arr):
    arr1 = [i for i in arr if i!='']
    return arr1
def preprocess(str):
    str = to_lower(str)
    str = rem_url(str)
    str = rem_username(str)
    str = rem_punc(str)
    str = rem_alphanum(str)
    str = rem_extra_alpha(str)
    str = rem_extraspace(str)
    #arr = tokenize(str)
    #rem_sw(arr)
    #spell_correct(arr)
    #arr = rem_empty(arr)
    #arr = lemmatize(arr)
    return str

In [ ]:
for i in range(len(df_train.index)):
    p_str = preprocess(df_train.iloc[i,1])
    df_train.at[i, 'tweet'] = p_str
for i in range(len(df_val.index)):
    p_str = preprocess(df_val.iloc[i,1])
    df_val.at[i, 'tweet'] = p_str

POS TAG


In [ ]:
def pos_encoding(df):
  sentags = [] #tags of all sentences
  pstindata = [] #pos tags in data
  from nltk.tokenize import word_tokenize
  result=[]
  postags = ['CC', 'CD', 'DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','SYM','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB', '$', '#', '``', '(',')',',','.',':' ]
  encodings=[]
  for t in df['tweet']:
    tagged = []
    result = word_tokenize(t)
    tagged = nltk.pos_tag(result)
    sentags.append(tagged)
    # for j in tagged:
    #   if(j[1] not in pstindata):
    #     pstindata.append(j[1])
    map={}
    encode=[]
    for j in tagged:
      if(j[1] in map):
        map[j[1]]+=1       #create map as tag, occurence for this sentence
      else:
        map[j[1]]=1
    for k in postags:
      if k in map:
        encode.append(map[k])
      else:
        encode.append(0)
    encodings.append(encode)
    for tags in map.keys():
      if(tags not in postags):
        print(tags, "found")
  return encodings
      #print(j[1])
  # print(postags)
  #print(len(encodings), len(encodings[0]))


In [ ]:
postags = ['CC', 'CD', 'DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']

In [ ]:
print(nltk.pos_tag)

In [ ]:
postags = ['CC', 'CD', 'DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']

NER

In [ ]:
import spacy
from spacy import displacy
#first counting the number of distinct ner tags present and then making dictionary so
# for all sentences the encoding represents the same order
NER = spacy.load("en_core_web_sm")
def ner_encoding(df):
    ner_arr = []
    for t in df['tweet']:
        ner_arr.append(NER(t))
    ner_encoding = []
    ner_dict = {}
    for ner_obj in ner_arr:
        for ent in ner_obj.ents:
            if not(ent.label_ in ner_dict.keys()):
                ner_dict.update({ent.label_:len(ner_dict)})
    import numpy as np
    for i in range(len(ner_arr)):
        temp = np.zeros((max(len(ner_dict),24),)) #there might be differences in
        #number of ner tags, hence we fix it to 24
        for ent in ner_arr[i].ents:
            temp[ner_dict[ent.label_]]+=1
        ner_encoding.append(temp)
    return ner_encoding

DEPENDENCY PARSING

In [ ]:
!pip install spacy
!pip install stanza
import spacy
import stanza
from spacy import displacy
from nltk.parse.stanford import StanfordDependencyParser
stanza.download('en')

In [ ]:
import pandas as pd
df = pd.read_csv('F1_Claim_Detection_train.csv')
dependency_parser = []
encoding = []
dictionaries = []
j =0
nlp = stanza.Pipeline('en', processors = "tokenize,mwt, lemma ,pos,depparse")
# list_temp = ["This is our NLP assignment", "Coronavirus was a world pandemic which caused deaths of many people."]
for tweet in df["tweet"]:
    j+=1
    #print(j)
    encod = nlp(tweet)

    store = encod.to_dict()

    encoding_sentence = []
    dictio = {}

    words = len(store[0])

    temp = []

    for i in range(words):
        dic_temp = store[0][i]
        temp.append(dic_temp["upos"])
    
    for ele in temp:
        if ele in dictio:
            dictio[ele] += 1
        else:
            dictio[ele] = 1

    dictionaries.append(dictio)
    # for key in dictio.keys():
    #     encoding_sentence.append(dictio[key])


    dependency_parser.append(encod)
    

tags = {}   # tags(dictionary) will contain all the possible upos values
for dic in dictionaries:

    for key in dic.keys():
        if key not in tags:
            tags[key] = 1
tags_list = [] # tags_list(array) will contain all the possible upos values
for key in tags.keys():
    tags_list.append(key)

for dic in dictionaries:
    # Each dic contain information about the tweets
    encoding_sentence = []
    for tag in tags_list:
        if tag in dic.keys():
            encoding_sentence.append(dic[tag])
        else:
            encoding_sentence.append(0)
    encoding.append(encoding_sentence)
# print(encoding, len(tags_list))
# print(len(tags))

df_encode = pd.DataFrame(encoding, columns=["1",'2','3','4','5','6','7','8','9', '10','11', '12', '13', '14', '15', '16', '17'])
df_encode.to_csv('encodings_train.csv', index=False)


In [ ]:
df_encode = pd.DataFrame(encoding, columns=["1",'2','3','4','5','6','7','8','9', '10','11', '12', '13', '14', '15', '16', '17'])
df_encode.to_csv('encodings_train.csv')

In [ ]:
import pandas as pd
df_test = pd.read_csv('F1_Claim_Detection_test.csv')
dependency_parser_test = []
encoding_test = []
dictionaries_test = []
j =0
nlp = stanza.Pipeline('en', processors = "tokenize,mwt, lemma ,pos,depparse")
# list_temp = ["This is our NLP assignment", "Coronavirus was a world pandemic which caused deaths of many people."]
for tweet in df_test["tweet"]:
    j+=1
    print(j)
    encod_test = nlp(tweet)

    store = encod_test.to_dict()

    encoding_sentence_test = []
    dictio_test = {}

    words = len(store[0])

    temp = []

    for i in range(words):
        dic_temp = store[0][i]
        temp.append(dic_temp["upos"])
    
    for ele in temp:
        if ele in dictio_test:
            dictio_test[ele] += 1
        else:
            dictio_test[ele] = 1

    dictionaries_test.append(dictio)
    # for key in dictio.keys():
    #     encoding_sentence.append(dictio[key])


    dependency_parser_test.append(encod)
    

tags_test = {}   # tags(dictionary) will contain all the possible upos values
for dic in dictionaries_test:

    for key in dic.keys():
        if key not in tags_test:
            tags_test[key] = 1
tags_list_test = [] # tags_list(array) will contain all the possible upos values
for key in tags_test.keys():
    tags_list_test.append(key)

for dic in dictionaries_test:
    # Each dic contain information about the tweets
    encoding_sentence_test = []
    for tag in tags_list_test:
        if tag in dic.keys():
            encoding_sentence_test.append(dic[tag])
        else:
            encoding_sentence_test.append(0)
    encoding_test.append(encoding_sentence_test)
# print(encoding, len(tags_list))
# print(len(tags))

df_encode_test = pd.DataFrame(encoding_test, columns=["1",'2','3','4','5','6','7','8','9'])
df_encode_test.to_csv('encodings_test.csv', index=False)


GLOVE

In [ ]:
import os
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/work/glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/glove')
import numpy as np
embeddings_index = {}
f = open('/content/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

def get_emb(tokenized_sent): #tokenized_sent is a 1d array
    vec = np.zeros((384,))
    count = 1
    for i in tokenized_sent:
        try:
            vec+=embeddings_index[i]
            count+=1
            
        except:
            pass
    vec/= count
    return vec

In [ ]:
glove_encoding_train = []
for t in df_train['tweet']:
    glove_encoding_train.append(get_emb(word_tokenize(t)))
"""glove_encoding_val = []
for t in df_val['tweet']:
    glove_encoding_val.append(get_emb(word_tokenize(t)))"""

BERT

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, losses, models, util

bert_model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
bert_emb_train = bert_model.encode(df_train['tweet'])
#print(bert_emb_train.shape)

In [ ]:
pos_encoding_train = pos_encoding(df_train)
ner_encoding_train = ner_encoding(df_train)
final_encoding_train = np.hstack(( ner_encoding_train, bert_emb_train))

In [ ]:
y_train = df_train['label'].values
y_val = df_train['label'].values

In [ ]:
from sklearn.svm import SVC
svm1 = SVC()
svm1.fit(final_encoding_train, y_train)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(final_encoding_train, y_train)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, max_depth=10)
rf.fit(final_encoding_train, y_train)

In [ ]:
print(y_train)

In [ ]:
pred_lr = lr.predict(final_encoding_val)
pred_svm = svm1.predict(final_encoding_val)
pred_rf = rf.predict(final_encoding_val)

In [ ]:
df_test = pd.read_csv('/content/F1_Claim_Detection_test.csv')

In [ ]:
for i in range(len(df_test.index)):
    p_str = preprocess(df_test.iloc[i,0])
    df_test.at[i, 'tweet'] = p_str

In [ ]:
pos_encoding_test = pos_encoding(df_test)
ner_encoding_test = ner_encoding(df_test)
"""glove_encoding_test = []
for t in df_test['tweet']:
    glove_encoding_test.append(get_emb(t))"""
bert_emb_test = bert_model.encode(df_test['tweet'])
print(bert_emb_test.shape)

In [ ]:
final_encoding_test = np.hstack((pos_encoding_test, ner_encoding_test, bert_emb_test))
print(np.shape(final_encoding_test))

In [ ]:
pred_lr = lr.predict(final_encoding_test)
pred_svm = svm1.predict(final_encoding_test)
pred_rf = rf.predict(final_encoding_test)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, pred_lr))

In [ ]:
pred = lr.predict(final_encoding_test)

In [ ]:
df_kaggle = pd.DataFrame(np.arange(0,len(pred)),columns = ['id'])

In [ ]:
df_kaggle.insert(1, "label", np.asarray(pred))

In [ ]:
df_kaggle

In [ ]:
df_kaggle.to_csv('/content/submission1.csv', index = False)

Oversampling/ Data augmentation

In [ ]:
!pip install numpy requests nlpaug
!pip install transformers


In [ ]:
import nlpaug.augmenter.word as naw
aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)
to_augment = df_train[df_train['label']==0]
to_augmentX = to_augment['tweet']
# to_augmentY = np.zeros(len(to_augmentX.index) * num_times, dtype=np.int8)
sentences = []
for j in to_augmentX:
  x = aug.augment(j,n=5)
  for sent in x:
    sentences.append(sent)
lbl=[]
for k in range (len(sentences)):
  lbl.append(0)

df2 = pd.DataFrame({"tweet":sentences,"label":lbl})
newdf_train = df_train.append(df2, ignore_index=True)
print(newdf_train)
newdf_train.to_csv('aug_train.csv', index=False,)  

Fine Tuning 

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [ ]:
!pip install datasets
!pip install transformers
from transformers import AutoTokenizer
import datasets
from datasets import load_dataset, load_from_disk
dataset = load_dataset('csv', data_files={'train': 'aug_train.csv'})

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")  #distilbert-base-uncased
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
checkpoint = "sentence-transformers/all-mpnet-base-v2"   #distilbert-base-uncased
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import TrainingArguments, Trainer
num_train_epochs=3
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=3)

In [ ]:
trainer = Trainer(args=training_args,model=model,
   train_dataset=tokenized_datasets["train"] 
)
trainer.train()
model.save_pretrained("claim/finetunemodel")

In [ ]:

# model.save_pretrained("claim/finetunemodel")
# alternatively save the trainer
# trainer.save_model("CustomModels/CustomHamSpam")
tokenizer.save_pretrained("claim/finetunemodel")
# load the model
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

load_model = AutoModelForSequenceClassification.from_pretrained("claim/finetunemodel")
load_tokenizer = AutoTokenizer.from_pretrained("claim/finetunemodel")
model = load_model
tokenizer = load_tokenizer
my_pipe  = pipeline("text-classification", model=load_model, tokenizer=load_tokenizer)
data = list(df_test["tweet"])


ls = my_pipe(data)
label_test=[]
for j in ls:
  if(j['label']=='LABEL_0'):
    label_test.append(0)
  elif(j['label']=='LABEL_1'):
    label_test.append(1)
  else:
    print("error")

df_kaggle = pd.DataFrame(np.arange(0,len(label_test)),columns = ['id'])
df_kaggle.insert(1, "label", np.asarray(label_test))
df_kaggle.to_csv('submission1.csv', index = False)

References for code:

[Extracting pre-trained GloVe embeddings](https://blog.paperspace.com/pre-trained-word-embeddings-natural-language-processing/)

[NER- Spacy](https://spacy.io/usage/linguistic-features#named-entities)

[Text augmentation](https://towardsdatascience.com/powerful-text-augmentation-using-nlpaug-5851099b4e97)

[Fine-tuning](https://levelup.gitconnected.com/how-to-fine-tune-an-nlp-classification-model-with-transformers-and-huggingface-1a2c0ea79c2)

Some code reused from previous assignments